In [1]:
%reload_ext autoreload
import sys
sys.path.append("../utils/")

# general imports
import matplotlib.pyplot as plt
%matplotlib inline

# AWS imports: Import Braket SDK modules

from braket.circuits import Circuit, Gate, Instruction, circuit, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
import numpy as np

import functools
import time

from quil_utils import *
from pyquil.quil import Pragma, Program
from pyquil.api import get_qc,QVM
from pyquil.gates import *
from quil_utils import Compiled_Circuit,Quil_to_Braket_Transpiler,Braket_to_Quil_Transpiler
from utils import DeviceUtils,DeviceScanner,BraketTaskScanner,Plotter
from IonQCompiler import IonQCompiler

import json
import pandas as pd
import seaborn as sns

In [2]:
scanner = BraketTaskScanner(task_arn='arn:aws:braket:us-east-1:058017682654:quantum-task/f0244b82-9c92-4d6b-a41a-bfe82c6335ad')

In [3]:
scanner.get_results()

[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', 

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

In [6]:
measurements_list = [['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '0', '0', '0'], ['1', '0', '1', '0', '0', '0', '0', '0', '0', '0'], ['0', '1', '1', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '0', '0', '0', '0'], ['0', '1', '1', '1', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '1', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0'], ['0', '0', '0', '1', '0', '1', '0', '0', '0', '0'], ['1', '0', '0', '0', '1', '1', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '1', '0', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '1', '0', '0', '0'], ['1', '0', '1', '0', '0', '0', '1', '0', '0', '0'], ['0', '1', '1', '0', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '0', '0', '1', '0', '0', '0'], ['0', '1', '1', '1', '0', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '1', '0', '1', '0', '0', '0'], ['0', '0', '0', '1', '1', '0', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '1', '1', '0', '0', '0'], ['0', '0', '0', '0', '0', '1', '1', '0', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '1', '0', '0'], ['0', '0', '1', '0', '0', '0', '0', '1', '0', '0'], ['1', '0', '1', '0', '0', '0', '0', '1', '0', '0'], ['1', '1', '1', '0', '0', '0', '0', '1', '0', '0'], ['0', '1', '0', '1', '0', '0', '0', '1', '0', '0'], ['0', '0', '1', '1', '0', '0', '0', '1', '0', '0'], ['0', '0', '0', '0', '0', '1', '0', '1', '0', '0'], ['0', '0', '0', '0', '0', '0', '1', '1', '0', '0'], ['0', '0', '0', '1', '0', '0', '1', '1', '0', '0'], ['0', '0', '0', '0', '0', '1', '1', '1', '0', '0'], ['0', '0', '1', '1', '0', '1', '1', '1', '0', '0'], ['0', '1', '0', '0', '1', '1', '1', '1', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '1', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '1', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '1', '0'], ['0', '0', '0', '0', '1', '0', '0', '0', '1', '0'], ['0', '0', '0', '0', '0', '1', '0', '0', '1', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0'], ['0', '0', '1', '0', '0', '0', '1', '0', '1', '0'], ['0', '0', '1', '0', '0', '0', '1', '0', '1', '0'], ['0', '0', '0', '1', '0', '0', '1', '0', '1', '0'], ['0', '0', '0', '0', '0', '1', '1', '0', '1', '0'], ['0', '0', '1', '0', '0', '1', '1', '0', '1', '0'], ['1', '0', '1', '0', '0', '1', '1', '0', '1', '0'], ['0', '0', '0', '1', '0', '1', '0', '1', '1', '0'], ['0', '0', '0', '0', '0', '0', '1', '1', '1', '0'], ['0', '0', '0', '1', '0', '0', '0', '0', '0', '1'], ['1', '1', '0', '1', '0', '0', '0', '0', '0', '1'], ['0', '1', '0', '0', '0', '0', '0', '1', '0', '1'], ['0', '0', '0', '0', '0', '1', '0', '1', '0', '1'], ['0', '0', '0', '0', '1', '1', '0', '1', '0', '1'], ['0', '0', '0', '0', '0', '0', '1', '1', '0', '1'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1'], ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1'], ['1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1'], ['0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1'], ['0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1'], ['0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '1'], ['0', '0', '1', '0', '0', '1', '1', '1', '0', '0', '1'], ['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '1'], ['0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1'], ['1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '1'], ['1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1'], ['1', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1'], ['0', '1', '0', '1', '0', '1', '1', '1', '0', '1', '1'], ['0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '1']]


In [7]:
np.asarray(measurements_list, dtype=int)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['1', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '1', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0', '1', '0', '0', '0', '0', '0', '0', '0',